In [1986]:
import pandas as pd
import sqlalchemy as sa
import requests
import json
import sqlalchemy
import nest_asyncio
from requests_html import AsyncHTMLSession

## CURRENT WEATHER API

In [1987]:
appid = '0ee1aeec809c143ac777fe7340414f60'
lat=43.6548
lon= -79.3883
units= 'metric'
limit = 10000
offset = 0

current_weather_api_url = 'https://api.openweathermap.org/data/2.5/weather?lat={}&lon={}&units={}&appid={}&limit={}&offset={}'.format(lat, lon, units, appid, limit, offset)

print(current_weather_api_url)

https://api.openweathermap.org/data/2.5/weather?lat=43.6548&lon=-79.3883&units=metric&appid=0ee1aeec809c143ac777fe7340414f60&limit=10000&offset=0


In [1988]:
api_response = requests.get(current_weather_api_url)

api_response

<Response [200]>

In [1989]:
data = api_response.json()
data

{'coord': {'lon': -79.3883, 'lat': 43.6548},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 28.84,
  'feels_like': 30.07,
  'temp_min': 25.31,
  'temp_max': 30.74,
  'pressure': 1004,
  'humidity': 55},
 'visibility': 10000,
 'wind': {'speed': 9.26, 'deg': 260},
 'clouds': {'all': 75},
 'dt': 1691869205,
 'sys': {'type': 1,
  'id': 718,
  'country': 'CA',
  'sunrise': 1691835513,
  'sunset': 1691886413},
 'timezone': -14400,
 'id': 6167863,
 'name': 'Downtown Toronto',
 'cod': 200}

In [1990]:
from datetime import datetime

In [1991]:
data_list = []

dt = pd.to_datetime(data['dt'], unit='s')
sys_sunrise = pd.to_datetime(data['sys']['sunrise'], unit='s')
sys_sunset = pd.to_datetime(data['sys']['sunset'], unit='s')

data_dict = {
    'timezone': data['timezone'],
    'id': data['id'],
    'name': data['name'],
    'coord_lon': data['coord']['lon'],
    'coord_lat': data['coord']['lat'],
    'cod': data['cod'],
    'weather_main': data['weather'][0]['main'],
    'weather_description': data['weather'][0]['description'],
    'weather_icon': data['weather'][0]['icon'],
    'base': data['base'],
    'main_temp': data['main']['temp'],
    'main_feels_like': data['main']['feels_like'],
    'main_temp_min': data['main']['temp_min'],
    'main_temp_max': data['main']['temp_max'],
    'main_pressure': data['main']['pressure'],
    'main_humidity': data['main']['humidity'],
    'visibility': data['visibility'],
    'wind_speed': data['wind']['speed'],
    'wind_deg': data['wind']['deg'],
    'clouds_all': data['clouds']['all'],
    'dt': dt,
    'sys_id': data['sys']['id'],
    'sys_country': data['sys']['country'],
    'sys_sunrise': sys_sunrise,
    'sys_sunset': sys_sunset
}

data_list.append(data_dict)

df = pd.DataFrame(data_list)

print(df)

   timezone       id              name  coord_lon  coord_lat  cod   
0    -14400  6167863  Downtown Toronto   -79.3883    43.6548  200  \

  weather_main weather_description weather_icon      base  ...  main_humidity   
0       Clouds       broken clouds          04d  stations  ...             55  \

   visibility  wind_speed  wind_deg  clouds_all                  dt  sys_id   
0       10000        9.26       260          75 2023-08-12 19:40:05     718  \

   sys_country         sys_sunrise          sys_sunset  
0           CA 2023-08-12 10:18:33 2023-08-13 00:26:53  

[1 rows x 25 columns]


In [1992]:
current_weather = pd.DataFrame(df)

print(current_weather)

   timezone       id              name  coord_lon  coord_lat  cod   
0    -14400  6167863  Downtown Toronto   -79.3883    43.6548  200  \

  weather_main weather_description weather_icon      base  ...  main_humidity   
0       Clouds       broken clouds          04d  stations  ...             55  \

   visibility  wind_speed  wind_deg  clouds_all                  dt  sys_id   
0       10000        9.26       260          75 2023-08-12 19:40:05     718  \

   sys_country         sys_sunrise          sys_sunset  
0           CA 2023-08-12 10:18:33 2023-08-13 00:26:53  

[1 rows x 25 columns]


In [1993]:
db_secret = {
    'drivername' : 'postgresql+psycopg2',
    'host'      : 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port'      : '5432',
    'username'  : 'mredshaw',
    'password'  : '2023!Schulich',
    'database'  : 'mredshaw_db'
}

In [1994]:
db_connection_url = sa.engine.URL.create(
    drivername = db_secret['drivername'],
    username   = db_secret['username'],
    password   = db_secret['password'],
    host       = db_secret['host'],
    port       = db_secret['port'],
    database   = db_secret['database']
)

In [1995]:
engine = sa.create_engine(db_connection_url)

In [1996]:
#with engine.connect() as connection:
#    connection.execute('CREATE SCHEMA IF NOT EXISTS uploads')

In [1997]:
current_weather

,timezone,id,name,coord_lon,coord_lat,cod,weather_main,weather_description,weather_icon,base,...,main_humidity,visibility,wind_speed,wind_deg,clouds_all,dt,sys_id,sys_country,sys_sunrise,sys_sunset
0,-14400,6167863,Downtown Toronto,-79.3883,43.6548,200,Clouds,broken clouds,04d,stations,...,55,10000,9.26,260,75,2023-08-12 19:40:05,718,CA,2023-08-12 10:18:33,2023-08-13 00:26:53


In [1998]:
current_weather.dtypes

timezone                        int64
id                              int64
name                           object
coord_lon                     float64
coord_lat                     float64
cod                             int64
weather_main                   object
weather_description            object
weather_icon                   object
base                           object
main_temp                     float64
main_feels_like               float64
main_temp_min                 float64
main_temp_max                 float64
main_pressure                   int64
main_humidity                   int64
visibility                      int64
wind_speed                    float64
wind_deg                        int64
clouds_all                      int64
dt                     datetime64[ns]
sys_id                          int64
sys_country                    object
sys_sunrise            datetime64[ns]
sys_sunset             datetime64[ns]
dtype: object

In [1999]:
current_weather.to_sql(
    name = 'current_weather', 
    schema = 'uploads',
    con = engine,
    if_exists = 'replace',
    index = False,
    method = 'multi',
      dtype = {
    'timezone': sa.types.INTEGER,
    'id': sa.types.INTEGER,
    'name': sa.types.VARCHAR(255),
    'coord_lon': sa.types.FLOAT,
    'coord_lat': sa.types.FLOAT,
    'cod': sa.types.INTEGER,
    'weather_main': sa.types.VARCHAR(255),
    'weather_description': sa.types.VARCHAR(255),
    'weather_icon': sa.types.VARCHAR(255),
    'base': sa.types.VARCHAR(255),
    'main_temp': sa.types.FLOAT,
    'main_feels_like': sa.types.FLOAT,
    'main_temp_min': sa.types.FLOAT,
    'main_temp_max': sa.types.FLOAT,
    'main_pressure': sa.types.INTEGER,
    'main_humidity': sa.types.INTEGER,
    'visibility': sa.types.INTEGER,
    'wind_speed': sa.types.FLOAT,
    'wind_deg': sa.types.INTEGER,
    'clouds_all': sa.types.INTEGER,
    'dt': sa.types.TIMESTAMP,
    'sys_id': sa.types.INTEGER,
    'sys_country': sa.types.VARCHAR(255),
    'sys_sunrise': sa.types.TIMESTAMP,  # Change this to TIMESTAMP
    'sys_sunset': sa.types.TIMESTAMP   # Change this to TIMESTAMP

})

1

## FORECAST WEATHER API

In [2000]:
appid = '0ee1aeec809c143ac777fe7340414f60'
lat=43.6548
lon= -79.3883
limit = 10000
offset = 0

forecast_weather_api_url = 'https://api.openweathermap.org/data/2.5/forecast?lat={}&lon={}&appid={}&limit={}&offset={}'.format(lat, lon, appid, limit, offset)

print(forecast_weather_api_url)

https://api.openweathermap.org/data/2.5/forecast?lat=43.6548&lon=-79.3883&appid=0ee1aeec809c143ac777fe7340414f60&limit=10000&offset=0


In [2001]:
api_response_forecast = requests.get(forecast_weather_api_url)

api_response_forecast

<Response [200]>

In [2002]:
forecast_data = api_response_forecast.json()
forecast_data

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1691874000,
   'main': {'temp': 301.99,
    'feels_like': 303.22,
    'temp_min': 300.9,
    'temp_max': 301.99,
    'pressure': 1004,
    'sea_level': 1004,
    'grnd_level': 995,
    'humidity': 55,
    'temp_kf': 1.09},
   'weather': [{'id': 803,
     'main': 'Clouds',
     'description': 'broken clouds',
     'icon': '04d'}],
   'clouds': {'all': 75},
   'wind': {'speed': 5.77, 'deg': 256, 'gust': 8.7},
   'visibility': 10000,
   'pop': 0.46,
   'sys': {'pod': 'd'},
   'dt_txt': '2023-08-12 21:00:00'},
  {'dt': 1691884800,
   'main': {'temp': 300.46,
    'feels_like': 301.39,
    'temp_min': 297.41,
    'temp_max': 300.46,
    'pressure': 1005,
    'sea_level': 1005,
    'grnd_level': 995,
    'humidity': 57,
    'temp_kf': 3.05},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10d'}],
   'clouds': {'all': 56},
   'wind': {'speed': 3.94, 'deg': 270, 'gust': 6.95},
   'visibili

In [2003]:
data_list = []

for item in forecast_data['list']:
    dt = pd.to_datetime(item['dt'], unit='s')
    main_temp = item['main']['temp']
    main_feels_like = item['main']['feels_like']
    temp_min = item['main']['temp_min']
    temp_max = item['main']['temp_max']
    pressure = item['main']['pressure']
    humidity = item['main']['humidity']
    
    weather_id = item['weather'][0]['id']
    weather_main = item['weather'][0]['main']
    weather_description = item['weather'][0]['description']
    weather_icon = item['weather'][0]['icon']
    
    data_list.append({
        'dt': dt,
        'main_temp': main_temp,
        'main_feels_like': main_feels_like,
        'temp_min': temp_min,
        'temp_max': temp_max,
        'pressure': pressure,
        'humidity': humidity,
        'weather_id': weather_id,
        'weather_main': weather_main,
        'weather_description': weather_description,
        'weather_icon': weather_icon
    })

forecast_df = pd.DataFrame(data_list)

print(forecast_df)


                    dt  main_temp  main_feels_like  temp_min  temp_max   
0  2023-08-12 21:00:00     301.99           303.22    300.90    301.99  \
1  2023-08-13 00:00:00     300.46           301.39    297.41    300.46   
2  2023-08-13 03:00:00     296.94           297.18    294.42    296.94   
3  2023-08-13 06:00:00     292.34           292.17    292.34    292.34   
4  2023-08-13 09:00:00     290.91           290.75    290.91    290.91   
5  2023-08-13 12:00:00     291.50           291.40    291.50    291.50   
6  2023-08-13 15:00:00     295.08           294.79    295.08    295.08   
7  2023-08-13 18:00:00     298.14           297.90    298.14    298.14   
8  2023-08-13 21:00:00     298.69           298.50    298.69    298.69   
9  2023-08-14 00:00:00     295.50           295.46    295.50    295.50   
10 2023-08-14 03:00:00     292.52           292.52    292.52    292.52   
11 2023-08-14 06:00:00     291.02           290.95    291.02    291.02   
12 2023-08-14 09:00:00     290.15     

In [2004]:
forecast_df.shape

(40, 11)

In [2005]:
forecast_weather = pd.DataFrame(forecast_df)

print(forecast_weather)

                    dt  main_temp  main_feels_like  temp_min  temp_max   
0  2023-08-12 21:00:00     301.99           303.22    300.90    301.99  \
1  2023-08-13 00:00:00     300.46           301.39    297.41    300.46   
2  2023-08-13 03:00:00     296.94           297.18    294.42    296.94   
3  2023-08-13 06:00:00     292.34           292.17    292.34    292.34   
4  2023-08-13 09:00:00     290.91           290.75    290.91    290.91   
5  2023-08-13 12:00:00     291.50           291.40    291.50    291.50   
6  2023-08-13 15:00:00     295.08           294.79    295.08    295.08   
7  2023-08-13 18:00:00     298.14           297.90    298.14    298.14   
8  2023-08-13 21:00:00     298.69           298.50    298.69    298.69   
9  2023-08-14 00:00:00     295.50           295.46    295.50    295.50   
10 2023-08-14 03:00:00     292.52           292.52    292.52    292.52   
11 2023-08-14 06:00:00     291.02           290.95    291.02    291.02   
12 2023-08-14 09:00:00     290.15     

In [2006]:
forecast_weather.dtypes

dt                     datetime64[ns]
main_temp                     float64
main_feels_like               float64
temp_min                      float64
temp_max                      float64
pressure                        int64
humidity                        int64
weather_id                      int64
weather_main                   object
weather_description            object
weather_icon                   object
dtype: object

In [2007]:
forecast_weather.to_sql(
    name = 'forecast_weather', 
    schema = 'uploads',
    con = engine,
    if_exists = 'replace',
    index = False,
    method = 'multi',
      dtype = {
          'dt' : sa.types.TIMESTAMP,
          'main_temp' : sa.types.FLOAT,
          'main_feels_like': sa.types.FLOAT,
          'temp_min': sa.types.FLOAT,
          'temp_max': sa.types.FLOAT,
          'pressure': sa.types.INTEGER,
          'humidity': sa.types.INTEGER,
          'weather_id': sa.types.INTEGER,
          'weather_main': sa.types.VARCHAR(255),
          'weather_description': sa.types.VARCHAR(255),
          'weather_icon': sa.types.VARCHAR(255)
})

40

## HISTORICAL WEATHER API

In [2008]:
key = '6fe878d1ff8d425185816caa82e1e9b8'
latitude= 43.6548
longitude= -79.3883
start_date = '2023-07-24'
end_date = '2023-08-07'
hourly = 'temperature_2m'
limit = 10000
offset = 0

history_weather_api_url2 = 'https://archive-api.open-meteo.com/v1/archive?latitude=43.6548&longitude=-79.3883&start_date=2023-07-24&end_date=2023-08-07&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration&timeformat=unixtime&timezone=America%2FNew_York'

print(history_weather_api_url2)

https://archive-api.open-meteo.com/v1/archive?latitude=43.6548&longitude=-79.3883&start_date=2023-07-24&end_date=2023-08-07&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration&timeformat=unixtime&timezone=America%2FNew_York


In [2009]:
history_api_response2 = requests.get(history_weather_api_url2)

history_api_response2

<Response [200]>

In [2010]:
history_data2 = history_api_response2.json()
history_data2

{'latitude': 43.699997,
 'longitude': -79.4,
 'generationtime_ms': 1.4420747756958008,
 'utc_offset_seconds': -14400,
 'timezone': 'America/New_York',
 'timezone_abbreviation': 'EDT',
 'elevation': 95.0,
 'daily_units': {'time': 'unixtime',
  'weathercode': 'wmo code',
  'temperature_2m_max': '°C',
  'temperature_2m_min': '°C',
  'temperature_2m_mean': '°C',
  'apparent_temperature_max': '°C',
  'apparent_temperature_min': '°C',
  'apparent_temperature_mean': '°C',
  'sunrise': 'unixtime',
  'sunset': 'unixtime',
  'precipitation_sum': 'mm',
  'rain_sum': 'mm',
  'snowfall_sum': 'cm',
  'precipitation_hours': 'h',
  'windspeed_10m_max': 'km/h',
  'windgusts_10m_max': 'km/h',
  'winddirection_10m_dominant': '°',
  'shortwave_radiation_sum': 'MJ/m²',
  'et0_fao_evapotranspiration': 'mm'},
 'daily': {'time': [1690171200,
   1690257600,
   1690344000,
   1690430400,
   1690516800,
   1690603200,
   1690689600,
   1690776000,
   1690862400,
   1690948800,
   1691035200,
   1691121600,
   16

In [2011]:

# Columns to extract
columns_to_extract = [
    'time',
    'weathercode',
    'temperature_2m_max',
    'temperature_2m_min',
    'temperature_2m_mean',
    'apparent_temperature_max',
    'apparent_temperature_min',
    'apparent_temperature_mean',
    'sunrise',
    'sunset',
    'precipitation_sum',
    'rain_sum',
    'snowfall_sum',
    'precipitation_hours',
    'windspeed_10m_max',
    'windgusts_10m_max',
    'winddirection_10m_dominant',
    'shortwave_radiation_sum',
    'et0_fao_evapotranspiration'
]

# Initialize empty dictionaries for each column
daily_data = {column: [] for column in columns_to_extract}

# Extract data from JSON
for column in columns_to_extract:
    daily_data[column] = history_data2['daily'][column]

# Create DataFrame
history_data2_df = pd.DataFrame(daily_data)
print(history_data2_df)


          time  weathercode  temperature_2m_max  temperature_2m_min   
0   1690171200         63.0                25.3                18.1  \
1   1690257600         51.0                25.3                17.8   
2   1690344000         63.0                27.8                20.7   
3   1690430400         63.0                27.6                21.7   
4   1690516800         51.0                28.6                19.3   
5   1690603200         63.0                23.6                17.9   
6   1690689600         61.0                22.7                15.8   
7   1690776000         51.0                22.6                16.8   
8   1690862400         51.0                23.1                15.7   
9   1690948800         51.0                24.2                16.0   
10  1691035200         61.0                27.2                19.8   
11  1691121600         51.0                25.2                18.9   
12  1691208000          3.0                23.3                17.6   
13  16

In [2012]:
history_data2_df

,time,weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
0,1690171200,63.0,25.3,18.1,21.9,27.0,19.7,23.9,1690192703,1690246182,4.5,4.5,0.0,6.0,15.3,37.1,239.0,21.46,3.89
1,1690257600,51.0,25.3,17.8,22.0,28.7,19.3,24.5,1690279164,1690332522,0.3,0.3,0.0,3.0,11.8,23.4,231.0,21.98,4.11
2,1690344000,63.0,27.8,20.7,24.2,29.5,21.8,25.7,1690365626,1690418860,6.4,6.4,0.0,4.0,22.1,47.2,213.0,23.19,4.89
3,1690430400,63.0,27.6,21.7,24.3,30.1,23.5,27.2,1690452089,1690505197,23.8,23.8,0.0,11.0,20.7,49.3,279.0,19.91,3.96
4,1690516800,51.0,28.6,19.3,24.4,32.5,21.2,27.3,1690538552,1690591532,0.2,0.2,0.0,1.0,14.5,32.0,239.0,23.22,4.58
5,1690603200,63.0,23.6,17.9,21.2,27.2,16.8,22.6,1690625016,1690677866,10.9,10.9,0.0,8.0,18.2,52.9,353.0,9.42,1.90
6,1690689600,61.0,22.7,15.8,19.6,24.2,16.2,20.2,1690711479,1690764198,3.2,3.2,0.0,8.0,10.9,24.1,299.0,20.33,3.75
7,1690776000,51.0,22.6,16.8,19.8,22.8,16.4,19.3,1690797944,1690850528,0.1,0.1,0.0,1.0,14.3,31.7,322.0,22.27,4.40
8,1690862400,51.0,23.1,15.7,19.7,23.6,15.7,19.8,1690884408,1690936857,0.1,0.1,0.0,1.0,11.0,18.4,321.0,20.65,3.81
9,1690948800,51.0,24.2,16.0,21.0,24.6,16.3,21.6,1690970873,1691023185,0.1,0.1,0.0,1.0,16.9,33.5,201.0,21.49,4.17


In [2013]:
history_data2_df.dtypes

time                            int64
weathercode                   float64
temperature_2m_max            float64
temperature_2m_min            float64
temperature_2m_mean           float64
apparent_temperature_max      float64
apparent_temperature_min      float64
apparent_temperature_mean     float64
sunrise                         int64
sunset                          int64
precipitation_sum             float64
rain_sum                      float64
snowfall_sum                  float64
precipitation_hours           float64
windspeed_10m_max             float64
windgusts_10m_max             float64
winddirection_10m_dominant    float64
shortwave_radiation_sum       float64
et0_fao_evapotranspiration    float64
dtype: object

In [2014]:
history_weather.to_sql(
    name='history_weather',
    schema='uploads',
    con=engine,
    if_exists='replace',
    index=False,
    method='multi',
    dtype={
        'time': sa.types.TIMESTAMP,
        'weathercode': sa.types.FLOAT,
        'temperature_2m_max': sa.types.FLOAT,
        'temperature_2m_min': sa.types.FLOAT,
        'temperature_2m_mean': sa.types.FLOAT,
        'apparent_temperature_max': sa.types.FLOAT,
        'apparent_temperature_min': sa.types.FLOAT,
        'apparent_temperature_mean': sa.types.FLOAT,
        'sunrise': sa.types.TIMESTAMP,
        'sunset': sa.types.TIMESTAMP,
        'precipitation_sum': sa.types.FLOAT,
        'rain_sum': sa.types.FLOAT,
        'snowfall_sum': sa.types.FLOAT,
        'precipitation_hours': sa.types.FLOAT,
        'windspeed_10m_max': sa.types.FLOAT,
        'windgusts_10m_max': sa.types.FLOAT,
        'winddirection_10m_dominant': sa.types.FLOAT,
        'shortwave_radiation_sum': sa.types.FLOAT,
        'et0_fao_evapotranspiration': sa.types.FLOAT
    }
)


220

In [2015]:
key = '617ed86c2036463688f42f2f808227bb'
lat=43.6548
lon= -79.3883
start_date = '2023-01-01:00'
end_date = '2023-08-09:00'
limit = 10000
offset = 0

history_weather_api_url = 'https://api.weatherbit.io/v2.0/history/daily?lat={}&lon={}&key={}&start_date={}&end_date={}&limit={}&offset={}'.format(lat, lon, key, start_date, end_date,limit, offset)

print(history_weather_api_url)

https://api.weatherbit.io/v2.0/history/daily?lat=43.6548&lon=-79.3883&key=617ed86c2036463688f42f2f808227bb&start_date=2023-01-01:00&end_date=2023-08-09:00&limit=10000&offset=0


In [2016]:
#history_api_response = requests.get(history_weather_api_url)

#history_api_response

In [2017]:
#history_data = history_api_response.json()
#history_data

In [2018]:
#history_data['data']

In [2019]:
#history_weather = pd.DataFrame(history_data['data'])
#history_weather

In [2020]:
#history_weather.dtypes

In [2021]:
#history_weather['datetime'] = pd.to_datetime(history_weather['datetime'])

In [2022]:
#history_weather.to_sql(
#    name='history_weather',
#    schema='uploads',
#    con=engine,
#    if_exists='replace',
#    index=False,
#    method='multi',
#    dtype={
#        'clouds': sa.types.INTEGER,
#        'datetime': sa.types.TIMESTAMP,
#        'dewpt': sa.types.FLOAT,
#        'dhi': sa.types.INTEGER,
#        'dni': sa.types.INTEGER,
#        'ghi': sa.types.INTEGER,
#        'max_dhi': sa.types.INTEGER,
#        'max_dni': sa.types.INTEGER,
#        'max_ghi': sa.types.INTEGER,
#        'max_temp': sa.types.FLOAT,
#        'max_temp_ts': sa.types.INTEGER,
#        'max_uv': sa.types.FLOAT,
#        'max_wind_dir': sa.types.INTEGER,
#        'max_wind_spd': sa.types.FLOAT,
#        'max_wind_spd_ts': sa.types.INTEGER,
#        'min_temp': sa.types.FLOAT,
#        'min_temp_ts': sa.types.INTEGER,
#        'precip': sa.types.FLOAT,
#        'precip_gpm': sa.types.FLOAT,
#        'pres': sa.types.INTEGER,
#        'revision_status': sa.types.VARCHAR(255),
#        'rh': sa.types.FLOAT,
#        'slp': sa.types.INTEGER,
#        'snow': sa.types.FLOAT,
#        'snow_depth': sa.types.FLOAT,
#        'solar_rad': sa.types.INTEGER,
#        't_dhi': sa.types.INTEGER,
#        't_dni': sa.types.INTEGER,
#        't_ghi': sa.types.INTEGER,
#        't_solar_rad': sa.types.INTEGER,
#        'temp': sa.types.FLOAT,
#        'ts': sa.types.INTEGER,
#        'wind_dir': sa.types.INTEGER,
#        'wind_gust_spd': sa.types.FLOAT,
#        'wind_spd': sa.types.FLOAT
#    }
#)

## WEATHER ALERT FETCH

In [2023]:
nest_asyncio.apply()
asession = AsyncHTMLSession()

In [2024]:
async def fetch_website(weburl):
    url = weburl
    r = await asession.get(url)
    
    if r.status_code == 200:
        print(f"Successfully fetched data from {url}")
    else:
        print(f"Failed to fetch data from {url}, status code: {r.status_code}")

    await r.html.arender(sleep=1)
    
    href_list = r.html.find('pre a[href]')
    href_values = [link.attrs['href'] for link in href_list]

    return href_values

In [2025]:
weburl = 'https://dd.weather.gc.ca/alerts/cap/'
href_values_list = asession.loop.run_until_complete(fetch_website(weburl))
latest_date = href_values_list[-1]

Successfully fetched data from https://dd.weather.gc.ca/alerts/cap/


In [2026]:
print("----------------------")
print("Today's Date : ",latest_date[0:4],"-",latest_date[4:6],"-",latest_date[6:8])
print("----------------------")
new_url = weburl+latest_date+'CWTO/'
print(new_url)

----------------------
Today's Date :  2023 - 08 - 12
----------------------
https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/


In [2027]:
new_href_values_list = asession.loop.run_until_complete(fetch_website(new_url))
latest_file = new_href_values_list[-1]

Successfully fetched data from https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/


In [2028]:
latest_file = new_href_values_list[-1]
print(latest_file)

19/


In [2029]:
newz_url = new_url+latest_file
print(newz_url)

https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/19/


In [2030]:
new_href_values_list = asession.loop.run_until_complete(fetch_website(newz_url))
latest_datz = new_href_values_list[-1]

Successfully fetched data from https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/19/


In [2031]:
latest_datz = new_href_values_list[-1]
print(latest_datz)

T_WZCN80_C_CWTO_202308121933_1408806460.cap


In [2032]:
latest_file = newz_url+latest_datz

In [2033]:
print(latest_file)

https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/19/T_WZCN80_C_CWTO_202308121933_1408806460.cap


In [2034]:
req = requests.get(latest_file)

In [2035]:
req.url

'https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/19/T_WZCN80_C_CWTO_202308121933_1408806460.cap'

In [2036]:
import os

directory_path = "/Users/mikeredshaw/Documents/Schulich MBAN/MMAI 5100 - Database Fundamentals/Weather API" # replace the file path
latest_datz = "example_file.xml" 
full_path = os.path.join(directory_path, latest_datz)

In [2037]:
with open(full_path, 'wb') as f:
    for chunk in req.iter_content(chunk_size=8192):
        if chunk:
            f.write(chunk)

In [2038]:
try:
    with open (full_path) as file:
        print("We can read the file John, Mike, Gabe, and Rahul")
except FileNotFoundError:
    print("I need help Boys!!")

We can read the file John, Mike, Gabe, and Rahul


In [2039]:
import xmltodict

with open(full_path, 'r') as xml_file:
    xml_data = xml_file.read()

In [2040]:
data_dict = xmltodict.parse(xml_data)

In [2041]:
import xmltodict

with open(full_path, 'r') as xml_file:
    xml_data = xml_file.read()

data_dict = xmltodict.parse(xml_data)

polygon_list = []


for info in data_dict['alert']['info']:
    if info.get('language') == 'en-CA':
        print("Language:", info['language'])
        print("Category:", info['category'])
        print("Event:", info['event'])
        print("Response Type:", info['responseType'])
        print("Urgency:", info['urgency'])
        print("Severity:", info['severity'])
        print("Certainty:", info['certainty'])
        print("Audience:", info['audience'])
        print("Effective:", info['effective'])
        print("Expires:", info['expires'])
        print("Sender Name:", info['senderName'])
        print("Headline:", info['headline'])
        print("Description:", info['description'])
        
        for area in info['area']:
            polygon = area['polygon']
            polygon_list.append(polygon)

Language: en-CA
Category: Met
Event: thunderstorm
Response Type: Monitor
Urgency: Expected
Severity: Severe
Certainty: Likely
Audience: general public
Effective: 2023-08-12T19:36:25-00:00
Expires: 2023-08-13T05:33:17-00:00
Sender Name: Environment Canada
Headline: severe thunderstorm watch in effect
Description: Conditions are favourable for the development of dangerous thunderstorms that may be capable of producing damaging wind gusts, damaging hail and heavy rain.

Hazards:
Toonie size hail.
Wind gusts up to 100 km/h.
Heavy rain.

Timing: 
This afternoon. 

Discussion:
Thunderstorms are expected to develop ahead of an approaching cold front this afternoon. The primary threat with these thunderstorms is hail and wind but heavy rain is also a threat. An isolated tornado also cannot be ruled out.

###

Very large hail can damage property, break windows, dent vehicles and cause serious injury. Very strong wind gusts can damage buildings, down trees and blow large vehicles off the road.



TypeError: string indices must be integers, not 'str'

In [ ]:
print("Polygon List:", polygon_list)

Polygon List: ['42.7799,-80.1221 42.7597,-80.2073 42.6893,-80.2824 42.6381,-80.3129 42.6437,-80.3844 42.6121,-80.4154 42.5491,-80.0252 42.5338,-80.0303 42.5205,-80.1237 42.5585,-80.3658 42.5635,-80.6074 42.5948,-80.7143 42.614,-80.713 42.6183,-80.7159 42.6227,-80.7182 42.6944,-80.7259 42.7882,-80.717 42.8071,-80.733 42.8238,-80.7446 42.8423,-80.7352 42.8493,-80.7272 42.8432,-80.7181 42.8389,-80.7156 42.8545,-80.7055 42.8738,-80.6048 42.8843,-80.5056 42.9282,-80.5357 42.9825,-80.5419 43.0171,-80.388 43.0355,-80.241 43.0306,-80.2122 43.0168,-80.1878 42.9663,-80.1666 42.9128,-80.1494 42.9105,-80.1653 42.9095,-80.1812 42.8521,-80.1507 42.7915,-80.1318 42.7879,-80.1269 42.7835,-80.1239 42.7799,-80.1221', '42.876,-79.4371 42.8607,-79.4301 42.8304,-79.4773 42.834,-79.5245 42.8191,-79.6066 42.8361,-79.6874 42.7846,-79.9766 42.798,-80.02 42.7835,-80.0541 42.7799,-80.1221 42.7835,-80.1239 42.7872,-80.1284 42.7915,-80.1318 42.8492,-80.161 42.9095,-80.1812 42.912,-80.1651 42.9128,-80.1494 42.9646,

In [ ]:
polygon_str = polygon_list[0]
points_str = polygon_str.split()

polygon_points = []

for point_str in points_str:
    lat, lon = map(float, point_str.split(','))
    polygon_points.append((lat, lon))

print(polygon_points)

[(42.7799, -80.1221), (42.7597, -80.2073), (42.6893, -80.2824), (42.6381, -80.3129), (42.6437, -80.3844), (42.6121, -80.4154), (42.5491, -80.0252), (42.5338, -80.0303), (42.5205, -80.1237), (42.5585, -80.3658), (42.5635, -80.6074), (42.5948, -80.7143), (42.614, -80.713), (42.6183, -80.7159), (42.6227, -80.7182), (42.6944, -80.7259), (42.7882, -80.717), (42.8071, -80.733), (42.8238, -80.7446), (42.8423, -80.7352), (42.8493, -80.7272), (42.8432, -80.7181), (42.8389, -80.7156), (42.8545, -80.7055), (42.8738, -80.6048), (42.8843, -80.5056), (42.9282, -80.5357), (42.9825, -80.5419), (43.0171, -80.388), (43.0355, -80.241), (43.0306, -80.2122), (43.0168, -80.1878), (42.9663, -80.1666), (42.9128, -80.1494), (42.9105, -80.1653), (42.9095, -80.1812), (42.8521, -80.1507), (42.7915, -80.1318), (42.7879, -80.1269), (42.7835, -80.1239), (42.7799, -80.1221)]


In [ ]:
def point_inside_polygon(x, y, poly):
    n = len(poly)
    inside = False
    p1x, p1y = poly[0]
    for i in range(n + 1):
        p2x, p2y = poly[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xints = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xints:
                        inside = 0
        p1x, p1y = p2x, p2y
    return 1


point_to_check = (43.6548, -79.3883)  

is_inside = point_inside_polygon(point_to_check[1], point_to_check[0], polygon_points)

if is_inside:
    print("The point is inside the polygon.")
else:
    print("The point is outside the polygon.")

The point is inside the polygon.
